In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.spatial import Voronoi
from shapely.geometry import Polygon

import sys
sys.path.append('../jamaica-infrastructure')

from scripts.preprocess.preprocess_utils \
import voronoi_finite_polygons_2d, extract_nodes_within_gdf,assign_value_in_area_proportions

In [ ]:
epsg=3448
node_id_column='id'
population_id_column='population'

# get sinks
nodes_dataframe = gpd.read_file('../data/spatial/infrasim-network/nodes.shp')
nodes_dataframe = nodes_dataframe[nodes_dataframe['asset_type'] == 'sink']
# get parish boundaries
parish_boundaries = gpd.read_file('../data/spatial/else/admin-boundaries.shp')
# get population dataframe
population_dataframe = gpd.read_file('../data/population-russell/population.gpkg')

# rename
nodes_dataframe['parish'] = nodes_dataframe['parish'].str.replace('Kingston','KSA')
nodes_dataframe['parish'] = nodes_dataframe['parish'].str.replace('St. Andrew','KSA')
parish_boundaries['Parish'] = parish_boundaries['Parish'].str.replace('KSA','KSA')
parish_boundaries['Parish'] = parish_boundaries['Parish'].str.replace('St','St.')

# change crs
nodes_dataframe = nodes_dataframe.to_crs(epsg=epsg)
parish_boundaries = parish_boundaries.to_crs(epsg=epsg)
population_dataframe = population_dataframe.to_crs(epsg=epsg)

combined_voronoi = []
combined_nodes = []

for unique_parish in nodes_dataframe.parish.unique():

    nodes_df = nodes_dataframe.loc[nodes_dataframe.parish.isin([unique_parish])].reset_index(drop=True).copy()
    parish_df = parish_boundaries.loc[parish_boundaries.Parish.isin([unique_parish])].reset_index(drop=True).copy()

    # load provinces and get geometry of the right population_dataframe
    sindex_parish_dataframe = parish_boundaries.sindex

    # create Voronoi polygons for the nodes
    xy_list = []
    for iter_, values in nodes_df.iterrows():
        xy = list(values.geometry.coords)
        xy_list += [list(xy[0])]

    vor = Voronoi(np.array(xy_list))
    regions, vertices = voronoi_finite_polygons_2d(vor)
    min_x = vor.min_bound[0] - 0.1
    max_x = vor.max_bound[0] + 0.1
    min_y = vor.min_bound[1] - 0.1
    max_y = vor.max_bound[1] + 0.1

    mins = np.tile((min_x, min_y), (vertices.shape[0], 1))
    bounded_vertices = np.max((vertices, mins), axis=0)
    maxs = np.tile((max_x, max_y), (vertices.shape[0], 1))
    bounded_vertices = np.min((bounded_vertices, maxs), axis=0)

    box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

    poly_list = []
    for region in regions:
        polygon = vertices[region]
        # Clipping polygon
        poly = Polygon(polygon)
        poly = poly.intersection(box)
        poly_list.append(poly)

    poly_index = list(np.arange(0, len(poly_list), 1))
    # make dataframe
    poly_df = pd.DataFrame(list(zip(poly_index, poly_list)),
                                columns=['gid', 'geometry'])
    # make geodataframe
    gdf_voronoi = gpd.GeoDataFrame(poly_df, geometry = 'geometry',crs=f'epsg:{epsg}')
    # add nodes
    gdf_voronoi[node_id_column] = gdf_voronoi.progress_apply(    
            lambda row: extract_nodes_within_gdf(row['geometry'],nodes_df,node_id_column),axis=1)
    # add parish
    gdf_voronoi['Parish'] = gdf_voronoi[node_id_column].map(nodes_df.set_index('id')['parish'].to_dict())
    # rename geom cols
    gdf_voronoi['voronoi_geom'] = gdf_voronoi['geometry']
    parish_df['parish_geom'] = parish_df['geometry']
    # dissolve
    gdf_voronoi = pd.merge(gdf_voronoi,parish_df[['Parish','parish_geom']],
                        how='left',on='Parish')
    # dropna/empty
    gdf_voronoi = gdf_voronoi[~gdf_voronoi.parish_geom.isna()].reset_index(drop=True)
    # perform spatial intersection
#     for i,row in gdf_voronoi.iterrows():
#         try:
#             (row.voronoi_geom.buffer(0)).intersection(row.parish_geom.buffer(0))
#         except:
#             print('Problem at index: ' + str(i))
#             print('>> Voronoi geom: ' + str(row.voronoi_geom))
#             print('>> Parish geom: ' + str(row.parish_geom))

    gdf_voronoi['geometry'] = gdf_voronoi.progress_apply(lambda row: \
                (row.voronoi_geom.buffer(0)).intersection(row.parish_geom.buffer(0)),axis=1)
    
    # append to master
    combined_voronoi.append(gdf_voronoi)
    combined_nodes.append(nodes_df)

# concat
voronois = gpd.GeoDataFrame( pd.concat( combined_voronoi, ignore_index=True) )
combined_nodes = gpd.GeoDataFrame( pd.concat( combined_nodes, ignore_index=True) ) 
# remove empty points
voronois = voronois[~voronois.geometry.is_empty].reset_index(drop=True)
# add population metrics
voronois[population_id_column] = 0
voronois = assign_value_in_area_proportions(population_dataframe, voronois, population_id_column)
voronois = voronois[~(voronois[node_id_column] == '')]
gdf_pops = voronois.copy()
# merge
nodes_dataframe = pd.merge(combined_nodes, gdf_pops, how='left', on=[node_id_column])
# save voronoi file
voronois['population'] = voronois[node_id_column].map(combined_nodes.set_index(node_id_column)['population'].to_dict())
voronois = voronois[['id','Parish','population','geometry']]
voronois = voronois[~voronois.geometry.is_empty].reset_index(drop=True)
#voronois.to_file(driver='ESRI Shapefile',filename='../data/spatial/infrasim-network/voronoi.shp')

In [ ]:
voronois